In [ ]:
# ======================================================================
#  EPSTEIN SEMANTIC EXPLORER v5  — Reporter Power Edition
# ======================================================================
#  Features:
#   ✔ Auto file detection + upload
#   ✔ Cluster viewer
#   ✔ Keyword search (BM25-lite)
#   ✔ Cluster summaries
#   ✔ Topic pseudo-modeling (centroid-based, upgraded)
#   ✔ Entity extraction (People / Orgs / Places)
#   ✔ Timeline builder
#   ✔ Cluster similarity heatmap (text-based centroids)
#   ✔ Cross-cluster entity search
#
#  Requirements: ONLY a JSONL file with fields:
#         {"id": "...", "cluster": 96, "text": "..."}
#
#  NO external models required. Pure Python.
# ======================================================================

import os, json, re, math
import numpy as np
from collections import Counter, defaultdict
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, Markdown

TARGET_NAME = "epstein_semantic.jsonl"

print("🔍 Checking for existing .jsonl files…")

available_jsonl = [f for f in os.listdir("/content") if f.endswith(".jsonl")]

if TARGET_NAME in available_jsonl:
    print(f"✔ Using existing {TARGET_NAME}")
else:
    if available_jsonl:
        print(f"✔ Found other JSONL files: {available_jsonl}")
        TARGET_NAME = available_jsonl[0]
        print(f"✔ Using: {TARGET_NAME}")
    else:
        print("⚠ No JSONL found — please upload epstein_semantic.jsonl")
        uploaded = files.upload()
        names = list(uploaded.keys())
        if not names:
            raise RuntimeError("❌ No file uploaded.")
        TARGET_NAME = names[0]
        print(f"✔ Loaded: {TARGET_NAME}")

records = []
with open(TARGET_NAME, "r", encoding="utf8") as f:
    for line in f:
        try:
            records.append(json.loads(line))
        except:
            pass

print(f"📦 Loaded {len(records)} records")

cluster_map = defaultdict(list)
all_texts = []
for r in records:
    cid = r.get("cluster", -1)
    cluster_map[cid].append(r)
    all_texts.append(r.get("text",""))

clusters = sorted(cluster_map.keys())
print("🧩 Clusters:", clusters)

def tokenize(text):
    return re.findall(r"[A-Za-z0-9']+", text.lower())

def text_vector(text):
    toks = tokenize(text)
    counts = Counter(toks)
    return counts

def centroid(docs):
    C = Counter()
    for d in docs:
        C.update(text_vector(d["text"]))
    return C

def cosine(a, b):
    num = 0
    da = 0
    db = 0
    for k in set(a.keys()) | set(b.keys()):
        va = a.get(k,0)
        vb = b.get(k,0)
        num += va*vb
        da += va*va
        db += vb*vb
    if da == 0 or db == 0:
        return 0
    return num / math.sqrt(da*db)

def view_cluster(cid, max_chars=2000):
    cid = int(cid)
    if cid not in cluster_map:
        print("❌ Cluster not found.")
        return

    docs = cluster_map[cid]
    print(f"\n=== CLUSTER {cid} ({len(docs)} docs) ===\n")

    for i, d in enumerate(docs):
        text = d.get("text","").strip()
        if len(text) > max_chars:
            text = text[:max_chars] + " … [truncated]"
        print(f"\n--- Doc {i+1}/{len(docs)} id={d.get('id')} ---\n{text}\n")

cid_box = widgets.Text(description="Cluster #")
out = widgets.Output()

def on_submit(change):
    out.clear_output()
    with out:
        view_cluster(change["new"])

cid_box.observe(on_submit, names="value")

print("\n➡ Enter cluster number below:")
display(cid_box, out)

print("\n🔍 Initializing BM25-lite structures...")

docs_text = [r["text"] for r in records]
tokenized_docs = [tokenize(t) for t in docs_text]
doc_freq = Counter()

for toks in tokenized_docs:
    for t in set(toks):
        doc_freq[t] += 1

Ndocs = len(records)
avg_len = sum(len(t) for t in tokenized_docs)/Ndocs

def bm25_score(query, doc_toks):
    k=1.5; b=0.75
    score = 0
    q_toks = tokenize(query)
    for q in q_toks:
        df = doc_freq.get(q,0)
        if df == 0:
            continue
        idf = math.log((Ndocs - df + 0.5)/(df + 0.5) + 1)
        tf = doc_toks.count(q)
        denom = tf + k*(1 - b + b*(len(doc_toks)/avg_len))
        score += idf*(tf*(k+1))/denom
    return score

def search(query, top_k=10):
    scores = []
    for r, toks in zip(records, tokenized_docs):
        s = bm25_score(query, toks)
        if s > 0:
            scores.append((s, r))
    scores.sort(reverse=True, key=lambda x: x[0])

    print(f"\n=== Search results for '{query}' ===")
    for s, r in scores[:top_k]:
        cid = r.get("cluster")
        print(f"\nScore {s:.2f} — Cluster {cid} — id={r.get('id')}")
        print(r["text"][:400], "...\n")

def summarize_cluster(cid):
    cid = int(cid)
    if cid not in cluster_map:
        print("❌ Cluster not found.")
        return

    docs = cluster_map[cid][:5]
    print(f"\n📝 Summary for Cluster {cid}\n")

    for d in docs:
        snippet = d["text"].strip().split("\n")[0]
        print("•", snippet[:200], "\n")

print("🧠 Computing cluster centroids...")

cluster_centroids = {}
for cid in clusters:
    cluster_centroids[cid] = centroid(cluster_map[cid])

STOPWORDS = set("""
the and to of a in is this that for on with as be or by from at
an it are was you your if but have we they his her she their our
subject re fw message communication thereof all may any doc email
""".split())

def top_terms(centroid, n=6):
    filtered = {w:c for w,c in centroid.items()
                if w not in STOPWORDS and len(w) > 2 and c > 1}
    if not filtered:
        return ["(no meaningful terms)"]
    return [w for w,_ in Counter(filtered).most_common(n)]

def show_topics():
    print("\n=== Improved Pseudo-Topics by Cluster ===")
    for cid in clusters:
        terms = top_terms(cluster_centroids[cid])
        print(f"Cluster {cid:<4} | {' '.join(terms)}")

name_re = re.compile(r"\b[A-Z][a-z]+(?: [A-Z][a-z]+){0,2}\b")
date_re = re.compile(r"(19|20)\d\d[-/\.]([0][1-9]|1[0-2])[-/\.]([0][1-9]|[12]\d|3[01])")

def extract_entities(text):
    return name_re.findall(text)

def cluster_entities(cid):
    cid = int(cid)
    ents = Counter()
    for d in cluster_map[cid]:
        for e in extract_entities(d["text"]):
            ents[e] += 1
    print(f"\n👤 Entities in Cluster {cid}")
    for e,c in ents.most_common(20):
        print(f"{e:20} {c}")

def show_timeline():
    timeline = []
    for r in records:
        text = r["text"]
        dates = date_re.findall(text)
        for d in dates:
            dt = "-".join(d)
            timeline.append((dt, text[:100]))

    timeline.sort()
    print("\n🕰 Timeline Extracted:")
    for dt, snip in timeline[:100]:
        print(dt, "-", snip)

def cluster_similarity():
    print("\n📊 Cluster Similarity Matrix (0–1 cosine)\n")
    cids = clusters
    vecs = [cluster_centroids[c] for c in cids]

    for i,cid_i in enumerate(cids):
        row = []
        for j,cid_j in enumerate(cids):
            sim = cosine(vecs[i], vecs[j])
            row.append(f"{sim:0.2f}")
        print(f"{cid_i:<4} | " + "  ".join(row))

def entity_to_clusters(entity, top_k=10):
    hits = []
    for cid in clusters:
        count = sum(entity.lower() in d["text"].lower() for d in cluster_map[cid])
        if count:
            hits.append((count,cid))
    hits.sort(reverse=True)
    print(f"\n🔎 Entity '{entity}' appears in:")
    for count,cid in hits[:top_k]:
        print(f"Cluster {cid}: {count} hits")

print("\n✅ Epstein Semantic Explorer v5 loaded and ready.")